<a href="https://colab.research.google.com/github/qhfks27/rani01/blob/master/Ch_04_%EC%8B%A0%EA%B2%BD%EB%A7%9D_%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/SeoulTechPSE/EngMath/blob/master/ch12.ipynb)

#신경망 학습
## 4.1 데이터에서 학습한다!
학습 : 훈련 데이터로 부터 가중치 매개변수의 최적값을 자동으로 획득하는 것

 - 데이터 → 사람이 생각한 특징(SIFT, HOG 등) → 기계학습(SVM, KNN 등) → 결과
 - 데이터 → 신경망(딥러닝) → 결과

범용 능력을 제대로 평가를 하기 위해서는 데이터를 훈련 데이터(training data)와 시험 데이터(test data)로 나눠 학습과 실험 수행  
오버피팅 :  한가지 데이터셋에만 지나치게 최적화된 상태

##4.2 손실 함수
손실 함수(loss function) : 신경망 학습에서 사용하는 지표, 신경망은 하나의 지표를 기준으로 최적의 매개변수 값을 탐색한다. 일반적으로 평균 제곱 오차와 교차 엔트로피 오차를 사용  

###4.2.1 평균 제곱 오차 (mean squared error, MSE)
$ E = \frac{1}{2} \sum_{k}^{ }(y_k - t_k)^2 $  
$y_k$ : 신경망의 출력 (신경망이 출력한 값)  
$t_k$ : 정답 레이블  
$k$ : 데이터 차원 수  

원-핫 인코딩 : 정답을 뜻하는 인수만 1, 나머지는 모두 0 인 배열


In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
import sys
sys.path.insert(0, '/content/gdrive/My Drive/Colab Notebooks/deep-learning-from-scratch-master/dataset')

from mnist import load_mnist

In [0]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from mnist import load_mnist

In [0]:
def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [6]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
mean_squared_error(np.array(y), np.array(t))

0.09750000000000003

In [7]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
mean_squared_error(np.array(y), np.array(t))

0.5975

###4.2.2 교차 엔트로피 오차(cross entropy error, CEE)
$ E = -\sum_{k}^{ }t_k \ln y_k  $   
$y_k$ : 신경망의 출력 (신경망이 출력한 값)  
$t_k$ : 정답 레이블 (정답에 해당하는 인덱스의 원소 : 1, 나머지 : 0 - 원-핫 인코딩) 

In [0]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))    #y=0일 경우, 마이너스 무한대를 뜻하는 -inf가 되어 계산 진행 안됨. 그래서 아주 작은값인 delta를 더해 마이너스 무한대가 발생하지 않도록 함.
  

In [10]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

In [11]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

2.302584092994546

###4.2.3 미니배치 학습
훈련 데이터에 대한 손실 함수의 값을 구하고, 그 값을 최대한 줄여주는 매개변수를 찾아낸다. 모든 훈련 데이터를 대상으로 손실 함수 값을 구해 손실 함수 값들의 합을 지표로 삼는다.   
$ E = - \frac{1}{N} \sum_{n}^{ } \sum_{k}^{} t_{nk}\log y_{nk} $  
$N$ : 데이터 개수   
$ t_{nk} $: n번째 데이터의 k번째 정답 레이블  
$ y_{nk} $: n번째 데이터의 k번째 신경망의 출력

미니배치(mini-batch) : 데이터가 너무 많을때 데이터 일부를 추려 전체의 '근사치'로 이용하고 신경망 학습에서도 훈련 데이터로부터 일부만 골라 학습을 수행하는데, 이때 이 일부를 미니배치라고 한다.

In [0]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from mnist import load_mnist

(x_train, t_train), (x_train, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)

In [13]:
print(x_train.shape)

(10000, 784)


In [14]:
print(t_train.shape)

(60000, 10)


In [0]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [16]:
np.random.choice(60000, 10) #0이상 60000미만 무작위로 10개 고르기

array([ 8854, 37916,  4156,  9624,  6747, 26972, 52876,  7544, 36171,
       35353])

###4.2.4 (배치용) 교차 엔트로피 오차 구현하기

In [0]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

$y$ : 신경망의 출력  
$t$ : 정답 레이블

$y$가 1차원이라면, 즉 데이터 하나당 교차 엔트로피 오차를 구하는 경우는 reshape 함수로 데이터의 형상을 바꿔준다. 배치의 크기로 나눠 정규화하고 이미지 1장당 평균의 교차 엔트로피 오차를 계산한다.

In [0]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size   #np.arange(batch_size)의 범위 : 0 ~ batch_size -1

batch_size가 5이면 np.arange(batch_size) ; [0, 1, 2, 3, 4] 넘파이 배열 형성  
$t$ ; [2, 7, 0, 9, 4]   
y[np.arange(batch_size), t] ; [y[0,2], y[1,7], y[2,0], y[3,9], y[4,4]]

###4.2.5 왜 손실 함수를 설정하는가?
  신경망을 학습할 때 정확도를 지표로 삼게 되면 매개변수의 미분이 대부분의 장소에서 0 이 된다. 가중치 매개변수의 값을 조금 바꾸더라도 거의 같은 값이며 연속적인 변화보다는 불연속적인 값으로 바뀌어 버린다.  
  손실함수를 지표로 삼을 경우, 미분 값이 음수면 양으로, 양이면 음수로 변화시켜 손실함수의 값을 줄일 수 있다. 매개변수의 값이 변할 경우 손실함수의 값은 연속적으로 변화한다.
  활성화 함수로 계단 함수를 이용할 경우 손실 삼후를 지표로 삼는 게 의미가 없어지기 때문에 시그모이드 함수와 같은 함수들을 활성화 함수로 쓰게 된다. 